# Low-Level Design (LLD) of a Vending Machine System

The Vending Machine System is an automated solution that allows users to select, pay for, and receive products, while managing inventory, payments, and refunds efficiently.

### 1. Requirements

1. **Product Selection**:
    - The system should allow a user to select a product from the available options.

2. **Payment Handling**:
    - The system should accept payments in specific denominations (e.g., coins and bills).

3. **Product Dispensing**:
    - Dispense the selected product after successful payment.

4. **Refund Handling**:
    - Refund the excess amount if overpaid or if the product cannot be dispensed.

5. **Inventory Management**:
    - Keep track of available products and quantities.

6. **Admin Operations**:
    - Allow restocking of products and cash collection from the machine.

---

### 2. Constraints

1. Fixed product inventory (e.g., 10 items per slot).
2. Limited denominations for payments (e.g., $1, $5, $10 coins/bills).
3. No change if the machine is out of coins

---

### 3. Identify Entities

1. **VendingMachine**:
    - Represents the entire vending machine system
    - Responsible for managing product selection, payments, and dispensing.

2. **Product**:
    - Represents an item in the vending machine.
    - Includes details like the name, price, and product ID.

3. **PaymentProcessor**:
    - Handles all payment-related operations, including accepting money, calculating refunds, and maintaining cash balance.

4. **Inventory**:
    - Manages the stock of products in the machine.
    - Tracks the quantity of each product and restocking operations.

5. **UserInterface**:
    - Provides interaction between the user and the vending machine.
    - Displays product options, handles input, and shows messages/errors.

### 4. Class Diagram

#### 4.1. Custom Exceptions

In [5]:
class VendingMachineException(Exception):
    """Base exception class for Vending Machine errors."""
    pass


class OutOfStockException(VendingMachineException):
    """Raised when the selected product is out of stock."""
    def __init__(self, product_name):
        super().__init__(f"'{product_name}' is out of stock.")


class InvalidProductException(VendingMachineException):
    """Raised when the selected product is invalid."""
    def __init__(self, product_id):
        super().__init__(f"Product with ID '{product_id}' does not exist.")


class InvalidAmountError(VendingMachineException):
    """Raised when the payment amount is invalid (less than or equal to zero)."""
    def __init__(self, message="Amount should be greater than zero."):
        self.message = message
        super().__init__(self.message)


class InvalidPaymentMethodException(VendingMachineException):
    """Raised when an unsupported payment method is provided."""
    def __init__(self, payment_method, message="Invalid payment method."):
        self.payment_method = payment_method
        self.message = f"{message} Supported methods: ['cash', 'card']"
        super().__init__(self.message)



#### 4.2. Product Class

In [7]:
class Product:
    def __init__(self, product_id, product_name, price, quantity):
        self.product_id = product_id
        self.product_name = product_name
        self.price = price
        self.quantity = quantity

    def __str__(self):
        return f"{self.product_name} - ${self.price} (Available: {self.quantity})"


#### 4.3. Inventory Class

In [9]:
class Inventory:

    def __init__(self):
        self.product_list = {}

    def add_product(self, product):
        """Add a product to the inventory."""
        self.product_list[product.product_id] = product

    def remove_product(self, product_id):
        """Remove a product from the inventory."""
        if product_id in self.product_list:
            del self.product_list[product_id]
        else:
            raise InvalidProductException(product_id)

    def update_quantity(self, product_id, quantity):
        """Update the quantity of a product."""
        if product_id in self.product_list:
            self.product_list[product_id].quantity = quantity
        else:
            raise InvalidProductException(product_id)

    def get_product_details(self, product_id):
        """Get details of a product."""
        if product_id in self.product_list:
            return self.product_list[product_id]
        else:
            raise InvalidProductException(product_id)

    def get_available_products(self):
        """Return a list of available products."""
        return [product for product in self.product_list.values() if product.quantity > 0]


#### 4.4. Payment Processor Class

In [11]:
class PaymentProcessor:
    def __init__(self):
        self.balance = 0
        self.payment_methods = ["cash", "card"]

    def process_payment(self, amount, payment_method):
        """Process the payment."""
        if amount <= 0:
            raise InvalidAmountException()

        if payment_method not in self.payment_methods:
            raise InvalidPaymentMethodException(payment_method)

        self.balance += amount

    def refund_amount(self, amount):
        """Refund the balance to the user."""
        if self.balance >= amount:
            self.balance -= amount
            return amount
        raise InsufficientBalanceException()


#### 4.5. UserInterface Class

In [13]:
class UserInterface:
    @staticmethod
    def display_message(message):
        """Display messages to the user."""
        print(message)

    @staticmethod
    def get_user_input(prompt):
        """Get user input from the console."""
        return input(prompt)



#### 4.7. Vending Machine Class 

In [15]:
class VendingMachine:

    def __init__(self):
        self.inventory = Inventory()
        self.payment_processor = PaymentProcessor()
        self.ui = UserInterface()
        self.current_balance = 0

    def select_product(self, product_id):
        """Allow user to select a product."""
        try:
            product = self.inventory.get_product_details(product_id)
            if product.quantity > 0:
                self.ui.display_message(f"Selected: {product.product_name}, Price: ${product.price}")
                return product
            else:
                self.ui.display_message("Product is out of stock.")
                return None
        except InvalidProductException:
            self.ui.display_message("Invalid product selection.")
            return None

    def insert_money(self, amount, payment_method="cash"):
        """Accept money and process payment."""
        try:
            self.payment_processor.process_payment(amount, payment_method)
            self.current_balance += amount
            self.ui.display_message(f"Inserted: ${amount}, Total balance: ${self.current_balance}")
        except ValueError as e:
            self.ui.display_message(str(e))

    def dispense_product(self, product):
        """Dispense the selected product if enough money is inserted."""
        if product is None:
            self.ui.display_message("No product selected.")
            return

        if self.current_balance >= product.price:
            change = self.current_balance - product.price
            refund = self.payment_processor.refund_amount(change)

            # Update inventory
            self.inventory.update_quantity(product.product_id, product.quantity - 1)
            self.current_balance = 0  # Reset balance

            self.ui.display_message(f"Dispensing: {product.product_name}")
            if refund > 0:
                self.ui.display_message(f"Refunding: ${refund}")
        else:
            self.ui.display_message("Insufficient balance. Please insert more money.")

    def refund(self):
        """Refund the current balance to the user."""
        if self.current_balance > 0:
            refund = self.payment_processor.refund_amount(self.current_balance)
            self.ui.display_message(f"Refunding: ${refund}")
            self.current_balance = 0
        else:
            self.ui.display_message("No balance to refund.")

    def restock_product(self, product, quantity):
        """Restock the vending machine with products."""
        self.inventory.add_product(product)
        self.inventory.update_quantity(product.product_id, quantity)
        self.ui.display_message(f"Restocked {quantity} of {product.product_name}.")


#### 5. Implementation

In [17]:
if __name__ == "__main__":
    vending_machine = VendingMachine()

    # Add products
    product1 = Product(1, "Soda", 2, 5)
    product2 = Product(2, "Chips", 3, 3)
    vending_machine.restock_product(product1, 5)
    vending_machine.restock_product(product2, 3)

    # User selects a product
    selected_product = vending_machine.select_product(1)

    # Insert money and buy the product
    if selected_product:
        vending_machine.insert_money(5, "cash")
        vending_machine.dispense_product(selected_product)


Restocked 5 of Soda.
Restocked 3 of Chips.
Selected: Soda, Price: $2
Inserted: $5, Total balance: $5
Dispensing: Soda
Refunding: $3
